In [6]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver/usr/bin

/bin/sh: apt-get: command not found
Unable to locate an executable at "/Library/Java/JavaVirtualMachines/jdk1.8.0_202.jdk/Contents/Home/bin/apt" (-1)
usage: cp [-R [-H | -L | -P]] [-fi | -n] [-apvXc] source_file target_file
       cp [-R [-H | -L | -P]] [-fi | -n] [-apvXc] source_file ... target_directory


In [7]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas
import json
import pprint
import numpy as np
import time

In [8]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [9]:
wd = webdriver.Chrome(executable_path='/Users/cangelosi/Desktop/chromedriver',chrome_options=chrome_options)
#wd.get("https://mutuionline.24oreborsaonline.ilsole24ore.com/")

/Users/cangelosi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [10]:
def parse_project(offerta_mutuo,list_data_extraction):
    chiave_ricerca=list_data_extraction
    rata = ""
    periodicita = ""
    banca = ""
    tipo_mutuo = ""
    tasso = ""
    spese_iniziali=""
    taeg=""
    link_dettagli=""
    vantaggi=""
    
    try:
        rata=offerta_mutuo.find_elements_by_css_selector("tbody > tr > td > div > span")[1].text

        periodicita=offerta_mutuo.find_elements_by_css_selector("tbody > tr > td > div > span")[2].text        
        
        banca=offerta_mutuo.find_elements_by_css_selector("tbody > tr:nth-child(1) > td:nth-child(3) > table > tbody > tr:nth-child(1) > td > div.prNameTitle > strong")[0].text
        comodo_tipo_mutuo=offerta_mutuo.find_elements_by_css_selector("tbody > tr > td > div > table > tbody > tr:nth-child(1) > td:nth-child(3) > table > tbody > tr:nth-child(1) > td > div.prNameTitle")[0].text
        tipo_mutuo=comodo_tipo_mutuo.lstrip(banca).lstrip("\n").lstrip(" ")
        tasso=offerta_mutuo.find_elements_by_css_selector("tbody > tr > td > table > tbody > tr:nth-child(2) > td.celladx-elencoprodotti > strong")[0].text    

        comodo_spese_iniziali=offerta_mutuo.find_elements_by_css_selector("tbody > tr > td > table > tbody > tr:nth-child(3) > td.celladx-elencoprodotti ")[0].text.split(sep='-')
        comodo_spese_iniziali2=[comodo_spese_iniziali[k].split(sep=':') for k in range(0,len(comodo_spese_iniziali))]
        spese_iniziali={k: v for k, v in comodo_spese_iniziali2}

        taeg=offerta_mutuo.find_elements_by_css_selector("tbody > tr > td > table > tbody > tr:nth-child(4) > td.celladx-elencoprodotti > strong")[0].text
        link_dettagli=offerta_mutuo.find_elements_by_css_selector("tbody > tr > td > table > tbody > tr:nth-child(5) > td > a")[0].get_attribute('href')
        comodo_vantaggi=offerta_mutuo.find_elements_by_css_selector("tbody > tr > td:nth-child(5) > table > tbody > tr > td:nth-child(2)")
        vantaggi=[comodo_vantaggi[j].text for j in range(0,len(comodo_vantaggi))]
    except:
        rata = "errore"
        periodicita="errore"
        banca = "errore"
        tipo_mutuo = "errore"
        tasso = "errore"
        spese_iniziali="errore"
        taeg="errore"
        link_dettagli="errore"
        vantaggi="errore"

    return {'chiave ricerca':chiave_ricerca,
            'rata': rata,
            'periodicita': periodicita,
            'banca': banca,
            'tipo_mutuo': tipo_mutuo,
            'tasso': tasso,
            'spese_iniziali': spese_iniziali,
            'taeg': taeg,
            'link_dettagli': link_dettagli,
            'vantaggi': vantaggi}



In [11]:
def bot_cerca_offerta_mutuo(num_iterazione,fin_mutuo_id, tipo_tasso_id, valore_immobile_id,valore_mutuo_value, durata_mutuo_id, eta_richiedente_value, pos_lavorativa_id, reddito_richiedente_value, comune_dom_id, com_immobile_id,stato_ricerca_id):
    finalita_mutuo_dominio=['Acquisto Prima Casa','Acquisto Seconda Casa','Surroga','Ristrutturazione']
    #Inserisco la finalità del mutuo
    valore_immobile = wd.find_element_by_id("CodiceFinalita_cb")
    valore_immobile.clear()
    time.sleep(0.8+np.random.randn()*0.2)    
    valore_immobile_value=finalita_mutuo_dominio[fin_mutuo_id]
    valore_immobile.send_keys(valore_immobile_value)
    valore_immobile.send_keys(u'\ue007')
    #Occorre capire se vada fatto invio dato che si tratta di menu combo

    time.sleep(3.2+np.random.randn()*0.2)

    tipo_tasso_dominio=['Fisso','Variabile','Variabile con cap','Variabile a rata costante']
    #Inserisco il tipo di tasso
    tipo_tasso = wd.find_element_by_id("CodiceTipoTasso_cb")
    tipo_tasso.clear()
    tipo_tasso_value=tipo_tasso_dominio[tipo_tasso_id]
    time.sleep(1)
    tipo_tasso.send_keys(tipo_tasso_value)
    tipo_tasso.send_keys(u'\ue007')

    time.sleep(3.3+np.random.randn()*0.2)

    #Inserisco il valore dell'immobile
    valore_immobile_dominio=[100000,150000,200000,250000,300000]
    valore_immobile = wd.find_element_by_id("Valore")
    valore_immobile.clear()
    valore_immobile_value=valore_immobile_dominio[valore_immobile_id]    
    time.sleep(2)    
    valore_immobile.send_keys(valore_immobile_value)
    valore_immobile.send_keys(u'\ue007')

    time.sleep(3.2+np.random.randn()*0.2)

    #Inserisco il valore del mutuo
    valore_mutuo = wd.find_element_by_id("ImportoMutuo")
    valore_mutuo.clear()
    #valore_mutuo_value=valore_mutuo
    valore_mutuo.send_keys(valore_mutuo_value)
    valore_mutuo.send_keys(u'\ue007')

    time.sleep(3.3+np.random.randn()*0.2)

    durata_mutuo_dominio=['5','7','10','12','15','20','25','30','35','40']
    #Durata mutuo
    durata_mutuo = wd.find_element_by_id("CodiceDurata_cb")
    durata_mutuo.clear()
    durata_mutuo_value=durata_mutuo_dominio[durata_mutuo_id]
    durata_mutuo.send_keys(durata_mutuo_value)
    durata_mutuo.send_keys(u'\ue007')

    time.sleep(3.2+np.random.randn()*0.2)

    #eta richiedente
    eta_richiedente = wd.find_element_by_name("Eta")
    eta_richiedente.clear()
    #eta_richiedente_value=eta
    eta_richiedente.send_keys(eta_richiedente_value)
    eta_richiedente.send_keys(u'\ue007')

    time.sleep(3.3+np.random.randn()*0.2)

    posizione_lavorativa_dominio=['Dip. tempo indeterminato','Dip. tempo determinato']
    #Posizione lavorativa
    posizione_lavorativa = wd.find_element_by_id("CodCategoriaReddituale_cb")
    posizione_lavorativa.clear()
    posizione_lavorativa_value=posizione_lavorativa_dominio[pos_lavorativa_id]
    posizione_lavorativa.send_keys(posizione_lavorativa_value)
    posizione_lavorativa.send_keys(u'\ue007')

    time.sleep(3.2+np.random.randn()*0.2)

    #reddito richiedente
    reddito_richiedente = wd.find_element_by_id("Reddito")
    reddito_richiedente.clear()
    #reddito_richiedente_value=reddito
    reddito_richiedente.send_keys(reddito_richiedente_value)
    reddito_richiedente.send_keys(u'\ue007')

    time.sleep(3.3+np.random.randn()*0.2)

    comune_domicilio_dominio=['Milano','Bergamo']
    #Comune di domicilio
    comune_domicilio = wd.find_element_by_id("ComuneResidenza")
    comune_domicilio.clear()
    comune_domicilio_value=comune_domicilio_dominio[comune_dom_id]
    comune_domicilio.send_keys(comune_domicilio_value)
    comune_domicilio.send_keys(u'\ue007')

    time.sleep(3.2+np.random.randn()*0.2)

    comune_immobile_dominio=['Milano','Bergamo']
    #Comune dell'immobile
    comune_immobile = wd.find_element_by_id("ComuneImmobile")
    comune_immobile.clear()
    comune_immobile_value=comune_immobile_dominio[com_immobile_id]
    comune_immobile.send_keys(comune_immobile_value)
    comune_immobile.send_keys(u'\ue007')

    time.sleep(3.3+np.random.randn()*0.2)

    stato_ricerca_immobile_dominio=['Firmato compromesso','Fatta proposta di acquisto','Individuato immobile']
    #Stato ricerca dell'immobile
    stato_ricerca_immobile = wd.find_element_by_id("CodStatoAvanzAcquisto_cb")
    stato_ricerca_immobile.clear()
    stato_ricerca_immobile_value=stato_ricerca_immobile_dominio[stato_ricerca_id]
    stato_ricerca_immobile.send_keys(stato_ricerca_immobile_value)
    durata_mutuo.send_keys(u'\ue007')


    time.sleep(3.2+np.random.randn()*0.2)

    #linkAggiornaRicerca <- Pulsante di aggiornamento ricerca

    #wd.save_screenshot('screenshot.png')

    #%pylab inline
    #import matplotlib.pyplot as plt
    #import matplotlib.image as mpimg
    #img=mpimg.imread('/content/screenshot.png')
    #imgplot = plt.imshow(img)
    #plt.show()

    #Effettuo il click sul form di ricerca
    #cerca_button=wd.find_element_by_css_selector('#div_submit > a')
    #time.sleep(4+np.random.randn()*0.2)
    if num_iterazione==1:
        wd.execute_script("$('#div_submit > a').click()")
    else:
        wd.execute_script("$('#linkAggiornaRicerca').click()")       

    #cerca_button.click
    time.sleep(10+np.random.randn()*0.2)
    flag_visualizzazione_unica = wd.find_element_by_css_selector("tr > td:nth-child(7) > div > label > i")
    flag_visualizzazione_unica.click()
    
    return 1

In [17]:
elenco_offerte_mutui = []
list_data_extraction=[[0,0,3,100000,6,30,0,2000,0,0,0],[0,0,4,100000,7,30,0,2500,0,0,0],[0,0,3,100000,6,30,0,2000,1,1,0],[0,0,4,100000,7,30,0,2500,1,1,0],[0,0,3,100000,6,30,0,3000,0,0,0],[0,0,4,100000,7,30,0,1500,1,1,0]]
wd.get("https://mutuionline.24oreborsaonline.ilsole24ore.com/")
time.sleep(30)

for i in tqdm(range(0,len(list_data_extraction))):
    bot_cerca_offerta_mutuo(i+1,list_data_extraction[i][0],\
                            list_data_extraction[i][1],\
                            list_data_extraction[i][2],\
                            list_data_extraction[i][3],\
                            list_data_extraction[i][4],\
                            list_data_extraction[i][5],\
                            list_data_extraction[i][6],\
                            list_data_extraction[i][7],\
                            list_data_extraction[i][8],\
                            list_data_extraction[i][9],\
                            list_data_extraction[i][10])    
    time.sleep(3)
    flag_visualizzazione_unica = wd.find_element_by_css_selector("tr > td:nth-child(7) > div > label > i")
    flag_visualizzazione_unica.click()
    time.sleep(5+np.random.randn()*0.2)
    wd.save_screenshot(f'screenshot_{i}.png')
    list_offerte_mutuo = wd.find_elements_by_css_selector("td > div > table")
              
    for offerta_mutuo_el in list_offerte_mutuo:
        elenco_offerte_mutui.append(parse_project(offerta_mutuo_el,list_data_extraction[i]))
              
              
              

/Users/cangelosi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [19]:
import pandas as pd
ds_elenco_mutui = pd.DataFrame(elenco_offerte_mutui)
#ds_elenco_mutui.set_index("project_id")
ds_elenco_mutui.head(70)
#ds_elenco_mutui.info()
ds_elenco_mutui.to_csv('ds_mutui.csv')

In [86]:
parse_project(list_offerte_mutuo[0])

{'rata': '€ 308,05',
 'periodicita': '(mensile)',
 'banca': 'BANCA CARIGE',
 'tipo_mutuo': 'MUTUO MIGLIORCASA FISSO',
 'tasso': 'Fisso: 0,70%',
 'spese_iniziali': {'Istruttoria': ' € 850,00 ', ' Perizia': ' € 270,00'},
 'taeg': '0,90%',
 'link_dettagli': 'https://mutuionline.24oreborsaonline.ilsole24ore.com/banche-mutui/banca-carige/mutuo-migliorcasa-fisso/21/71/?22837226551497063662',
 'vantaggi': ['Condizioni esclusive MutuiOnline']}

In [84]:
list_offerte_mutuo = wd.find_elements_by_css_selector("td > div > table")
offerta_mutuo=list_offerte_mutuo[6]
comodo_vantaggi=offerta_mutuo.find_elements_by_css_selector("tbody > tr > td:nth-child(5) > table > tbody > tr > td:nth-child(2)")
vantaggi=[comodo_vantaggi[j].text for j in range(0,len(comodo_vantaggi))]
print(vantaggi)


['Inizi a pagare dopo 1 anno', 'Tasso Promozionale con sottoscrizione Polizza Vita', 'Extra sconto 0,10% per classe energetica A o B']
